# Linear regression with the Associated Press

In [this piece from the Associated Press](https://apnews.com/66ac44186b6249709501f07a7eab36da), Nicky Forster combines from the US Census Bureau and the CDC to see how life expectancy is related to actors like unemployment, income, and others. We'll be looking at how they can write sentences like this one: 

> "An increase of 10 percentage points in the unemployment rate in a neighborhood translated to a loss of roughly a year and a half of life expectancy, the AP found."

## Our datasets

* **R12221544_SL140.csv:** Employment status by census tract - ACS 2015 5-year, tract level, table B23025
* **R12221544.txt:** Data dictionary for R12221544_SL140.csv - Description of columns and tables
* **R12221550_SL140.csv:** Demographic data by census tract - ACS 2015 5-year, tract level, tables B23025, B06009, B03002, B19013, C17002
* **R12221550.txt:** Data dictionary for R12221550_SL140.csv - Description of columns and tables
* **US_A.CSV:** USALEEP national data - estimates of life expectancy at birth for most of the census tracts in the USA
* **Record_Layout_CensusTract_Life_Expectancy.pdf:** USALEEP data dictionary - None


In [37]:
import pandas as pd
import numpy as np

# Keep decimal numbers to 4 decimal places
pd.set_option("display.float_format", '{:,.4f}'.format)
pd.set_option("display.max_columns", 100)

## Reading in our data

We'll start by reading in our datasets (as we must!).

### Life expectancy data

The first dataset is [USALEEP](https://www.cdc.gov/nchs/nvss/usaleep/usaleep.html), the U.S. Small-area Life Expectancy Estimates Project. It's from the National Center for Health Statistics at the CDC.

We're going to rename a few columns so they make a little more sense. The columns themselves aren't too complicated, mostly pointing out the census tracts and the life expectancy.

In [38]:
life_expec = pd.read_csv("US_A.CSV")
life_expec.columns = ['tract_id', 'STATE2KX','CNTY2KX', 'TRACT2KX', 'life_expectancy', 
                      'life_expectancy_std_err', 'flag']
life_expec.head()

,tract_id,STATE2KX,CNTY2KX,TRACT2KX,life_expectancy,life_expectancy_std_err,flag
0,1001020100,1,1,20100,73.1000,2.2348,3
1,1001020200,1,1,20200,76.9000,3.3453,3
2,1001020400,1,1,20400,75.4000,1.0216,3
3,1001020500,1,1,20500,79.4000,1.1768,1
4,1001020600,1,1,20600,73.1000,1.5519,3


`flag` sounds a little mysterious and scary, but it's just whether the numbers were observed or predicted.

### Unemployment Data

For unemployment, we'll be using data from the Census. You can get unemployment measures from a lot of places in the US government, so the Census Bureau actually [publishes data on which ones you should use](https://www.census.gov/topics/employment/labor-force/guidance.html). In our case we're going to collect unemployment data from the American Community Survery, for the 5-year period nearest to our life expectancy dataset.

We'll be using Table B23025005, Employment Status for the Population 16 Years and Over. Take a look at [the codebook](https://www.socialexplorer.com/data/ACS2015_5yr/metadata/?ds=ACS15_5yr&var=B23025005) for more details. We're only reading in a few columns to keep things looking clean.

In [39]:
employment = pd.read_csv("R12221544_SL140.csv", encoding='latin-1')
employment.head(2)

,Geo_FIPS,Geo_GEOID,Geo_NAME,Geo_QName,Geo_STUSAB,Geo_SUMLEV,Geo_GEOCOMP,Geo_FILEID,Geo_LOGRECNO,Geo_US,Geo_REGION,Geo_DIVISION,Geo_STATECE,Geo_STATE,Geo_COUNTY,Geo_COUSUB,Geo_PLACE,Geo_PLACESE,Geo_TRACT,Geo_BLKGRP,Geo_CONCIT,Geo_AIANHH,Geo_AIANHHFP,Geo_AIHHTLI,Geo_AITSCE,Geo_AITS,Geo_ANRC,Geo_CBSA,Geo_CSA,Geo_METDIV,Geo_MACC,Geo_MEMI,Geo_NECTA,Geo_CNECTA,Geo_NECTADIV,Geo_UA,Geo_UACP,Geo_CDCURR,Geo_SLDU,Geo_SLDL,Geo_VTD,Geo_ZCTA3,Geo_ZCTA5,Geo_SUBMCD,Geo_SDELM,Geo_SDSEC,Geo_SDUNI,Geo_UR,Geo_PCI,Geo_TAZ,Geo_UGA,Geo_BTTR,Geo_BTBG,Geo_PUMA5,Geo_PUMA1,ACS15_5yr_B23025001,ACS15_5yr_B23025002,ACS15_5yr_B23025003,ACS15_5yr_B23025004,ACS15_5yr_B23025005,ACS15_5yr_B23025006,ACS15_5yr_B23025007,ACS15_5yr_B23025001s,ACS15_5yr_B23025002s,ACS15_5yr_B23025003s,ACS15_5yr_B23025004s,ACS15_5yr_B23025005s,ACS15_5yr_B23025006s,ACS15_5yr_B23025007s
0,1001020100,14000US01001020100,"Census Tract 201, Autauga County, Alabama","Census Tract 201, Autauga County, Alabama",al,140,0,ACSSF,1760,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,20100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1554,997,997,943,54,0,557,92.1212,85.4545,85.4545,83.6364,18.7879,6.6667,67.8788
1,1001020200,14000US01001020200,"Census Tract 202, Autauga County, Alabama","Census Tract 202, Autauga County, Alabama",al,140,0,ACSSF,1761,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,20200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1731,884,869,753,116,15,847,143.0303,115.1515,114.5455,107.2727,38.1818,14.5455,86.6667


In [40]:
# Importing again the same file, but only certain columns. Also rename it.
columns = {
    'Geo_FIPS': 'Geo_FIPS',
    'ACS15_5yr_B23025001': 'total_pop',
    'ACS15_5yr_B23025005': 'unemployed'
}
employment = pd.read_csv("R12221544_SL140.csv", usecols=columns.keys(), encoding='latin-1')

employment = employment.rename(columns=columns)
employment.head()

,Geo_FIPS,total_pop,unemployed
0,1001020100,1554,54
1,1001020200,1731,116
2,1001020300,2462,91
3,1001020400,3424,216
4,1001020500,8198,221


### Demographic data

The other dataset we're using is also from the Census Bureau's American Community Survey. It's a collection of many more columns with very, very long names and very, very mysterious codes. The tables include:

* [B03002](https://censusreporter.org/tables/B03002/): Hispanic or Latino Origin by Race
* [B06009](https://censusreporter.org/tables/B06009/): Place of Birth by Educational Attainment in the United States
* [C17002](https://censusreporter.org/tables/C17002/): Ratio of Income to Poverty Level
* [B19013](https://censusreporter.org/tables/B19013/): Median Household Income

Again, we're only picking a few columns to read in. Generally they're the raw count of certain populations, as well as the total population counted of each table.

In [41]:
columns = {
    'Geo_FIPS': 'Geo_FIPS',
    'ACS15_5yr_B03002001': 'race_table_total',
    'ACS15_5yr_B03002004': 'black',
    'ACS15_5yr_B03002003': 'white',
    'ACS15_5yr_B03002012': 'hispanic',
    'ACS15_5yr_B06009001': 'edu_table_total',
    'ACS15_5yr_B06009002': 'less_than_hs',
    'ACS15_5yr_C17002004': 'poverty_100_124',
    'ACS15_5yr_C17002005': 'poverty_125_149',
    'ACS15_5yr_C17002001': 'poverty_table_total',
    'ACS15_5yr_B19013001': 'income'
}        
census = pd.read_csv("R12221550_SL140.csv", usecols=columns.keys(), encoding='latin-1')
census = census.rename(columns=columns)
census.head()

,Geo_FIPS,race_table_total,white,black,hispanic,edu_table_total,less_than_hs,poverty_table_total,poverty_100_124,poverty_125_149,income
0,1001020100,1948,1703,150,17,"1,243.0000",184.0000,1948,81,101,"61,838.0000"
1,1001020200,2156,872,1149,17,"1,397.0000",356.0000,1983,232,58,"32,303.0000"
2,1001020300,2968,2212,551,0,"2,074.0000",221.0000,2968,148,207,"44,922.0000"
3,1001020400,4423,3662,162,464,"2,899.0000",339.0000,4423,141,182,"54,329.0000"
4,1001020500,10763,7368,2674,80,"6,974.0000",310.0000,10563,256,1064,"51,965.0000"


## Merge the data

To perform our regression, we need all of our data in **a single dataframe**. Fortunately these dataframes all keep track of census tracks with [FIPS codes](https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt), unique geographic codes that we can rely on to merge our datasets.

In [42]:
merged = life_expec.merge(employment, left_on='tract_id', right_on='Geo_FIPS')
merged = merged.merge(census, left_on='Geo_FIPS', right_on='Geo_FIPS')
merged.head()

,tract_id,STATE2KX,CNTY2KX,TRACT2KX,life_expectancy,life_expectancy_std_err,flag,Geo_FIPS,total_pop,unemployed,race_table_total,white,black,hispanic,edu_table_total,less_than_hs,poverty_table_total,poverty_100_124,poverty_125_149,income
0,1001020100,1,1,20100,73.1000,2.2348,3,1001020100,1554,54,1948,1703,150,17,"1,243.0000",184.0000,1948,81,101,"61,838.0000"
1,1001020200,1,1,20200,76.9000,3.3453,3,1001020200,1731,116,2156,872,1149,17,"1,397.0000",356.0000,1983,232,58,"32,303.0000"
2,1001020400,1,1,20400,75.4000,1.0216,3,1001020400,3424,216,4423,3662,162,464,"2,899.0000",339.0000,4423,141,182,"54,329.0000"
3,1001020500,1,1,20500,79.4000,1.1768,1,1001020500,8198,221,10763,7368,2674,80,"6,974.0000",310.0000,10563,256,1064,"51,965.0000"
4,1001020600,1,1,20600,73.1000,1.5519,3,1001020600,2855,190,3851,2808,459,503,"2,356.0000",412.0000,3851,212,206,"63,092.0000"


# ALWAYS NORMALIZE; DONT USE RAW NUMBERS


In [46]:
merged['pct_unemployed'] = merged.unemployed/merged.total_pop * 100
merged['pct_less_than_hs'] = merged.less_than_hs/merged.total_pop * 100
merged['pct_white'] = merged.white / merged.race_table_total * 100

merged.head(2)

,tract_id,STATE2KX,CNTY2KX,TRACT2KX,life_expectancy,life_expectancy_std_err,flag,Geo_FIPS,total_pop,unemployed,race_table_total,white,black,hispanic,edu_table_total,less_than_hs,poverty_table_total,poverty_100_124,poverty_125_149,income,pct_unemployed,pct_less_than_hs,pct_white
0,1001020100,1,1,20100,73.1000,2.2348,3,1001020100,1554,54,1948,1703,150,17,"1,243.0000",184.0000,1948,81,101,"61,838.0000",3.4749,11.8404,87.4230
1,1001020200,1,1,20200,76.9000,3.3453,3,1001020200,1731,116,2156,872,1149,17,"1,397.0000",356.0000,1983,232,58,"32,303.0000",6.7013,20.5661,40.4453


## Running the regression

Using the `statsmodels` package, we'll run a linear regression to find the relationship between **life expectancy** and our **columns**. 

> Python note: I'm also using a [multiline string](https://stackoverflow.com/posts/10660443/revisions) which is started and ended using three quotation marks. Multi-line strings and formulas are a match made in heaven!

In [47]:
import statsmodels.formula.api as smf

model = smf.ols("""
    life_expectancy ~ pct_unemployed + pct_less_than_hs + pct_white
    """, data=merged)

results = model.fit()
results.summary()



# R-squared: how much % of variance is explained by these factors

# If my unemployment rate goes up by 1 percentage point, my life expectancy will lower by roughly four months.

# If the unemployment rate goes up by 1 percentage point,

# life expectancy will drop by 0.376 years.

# life expectancy will drop by four and a half months.



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        life_expectancy   R-squared:                       0.248
Model:                            OLS   Adj. R-squared:                  0.248
Method:                 Least Squares   F-statistic:                     7235.
Date:                Tue, 10 Aug 2021   Prob (F-statistic):               0.00
Time:                        23:57:10   Log-Likelihood:            -1.7481e+05
No. Observations:               65662   AIC:                         3.496e+05
Df Residuals:                   65658   BIC:                         3.497e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           83.7978      0.064   1304.843      0.000      83.672      83.924
pct_unemployed      -0.4370      0.005    -86.881      0.000      -0.447      -0.427
pct_less_than_hs    -0.1568      0.002    -83.003      0.000      -0.160      -0.153
pct_white           -0.0200      0.001    -33.876      0.000      -0.021      -0.019
==============================================================================
Omnibus:                      810.878   Durbin-Watson:                   1.177
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1446.483
Skew:                          -0.042   Prob(JB):                         0.00
Kurtosis:                       3.722   Cond. No.                         333.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [52]:
# The increase of $1 dollar is too small, so we will divide it
model = smf.ols("""
    life_expectancy ~ pct_unemployed + pct_less_than_hs + pct_white 
    + np.divide(income,25000)
    """, data=merged)

results = model.fit()

results.summary()

# R-squared: how much % of variance is explained by these factors

# For an increase of 25000 we expect an increase of almost 2 yrs


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        life_expectancy   R-squared:                       0.402
Model:                            OLS   Adj. R-squared:                  0.402
Method:                 Least Squares   F-statistic:                 1.104e+04
Date:                Wed, 11 Aug 2021   Prob (F-statistic):               0.00
Time:                        00:13:42   Log-Likelihood:            -1.6728e+05
No. Observations:               65656   AIC:                         3.346e+05
Df Residuals:                   65651   BIC:                         3.346e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   76.7315      0.079    971.350      0.000      76.577      76.886
pct_unemployed              -0.2698      0.005    -57.768      0.000      -0.279      -0.261
pct_less_than_hs            -0.0334      0.002    -17.257      0.000      -0.037      -0.030
pct_white                   -0.0096      0.001    -17.930      0.000      -0.011      -0.009
np.divide(income, 25000)     1.7428      0.013    129.926      0.000       1.717       1.769
==============================================================================
Omnibus:                     1370.319   Durbin-Watson:                   1.361
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2895.946
Skew:                           0.082   Prob(JB):                         0.00
Kurtosis:                       4.016   Cond. No.                         463.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""